Installing SoccerNet

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [9]:
!pip install SoccerNet

     |████████████████████████████████| 68 kB 3.2 MB/s 
     |████████████████████████████████| 2.3 MB 8.4 MB/s 


Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import logging
import os
import zipfile
import sys
import json
import time
from tqdm import tqdm
import torch
import numpy as np


Extracting and Preprocessing Dataset

In [2]:
from torch.utils.data import Dataset

import numpy as np
import random
import os
import time

import sklearn
import sklearn.metrics
from sklearn.metrics import average_precision_score
from SoccerNet.Evaluation.ActionSpotting import evaluate
from tqdm import tqdm

import torch

import logging
import json

from SoccerNet.Downloader import getListGames
from SoccerNet.Downloader import SoccerNetDownloader
from SoccerNet.Evaluation.utils import AverageMeter, EVENT_DICTIONARY_V2, INVERSE_EVENT_DICTIONARY_V2
from SoccerNet.Evaluation.utils import EVENT_DICTIONARY_V1, INVERSE_EVENT_DICTIONARY_V1



def feats2clip(feats, stride, clip_length, padding = "replicate_last", off=0):
    if padding =="zeropad":
        print("beforepadding", feats.shape)
        pad = feats.shape[0] - int(feats.shape[0]/stride)*stride
        print("pad need to be", clip_length-pad)
        m = torch.nn.ZeroPad2d((0, 0, clip_length-pad, 0))
        feats = m(feats)
        print("afterpadding", feats.shape)
        # nn.ZeroPad2d(2)

    idx = torch.arange(start=0, end=feats.shape[0]-1, step=stride)
    idxs = []
    for i in torch.arange(-off, clip_length-off):
        idxs.append(idx+i)
    idx = torch.stack(idxs, dim=1)

    if padding=="replicate_last":
        idx = idx.clamp(0, feats.shape[0]-1)
    # print(idx)
    return feats[idx,...]


class SoccerNetClips(Dataset):
    def __init__(self, path, features="baidu_soccer_embeddings.npy", split=["train"], version=2, 
                framerate=1, window_size=10):
        self.path = path
        self.listGames = getListGames(split)[:50]
        self.features = features
        self.window_size_frame = window_size*framerate
        self.version = version
        if version == 1:
            self.num_classes = 3
            self.labels="Labels.json"
        elif version == 2:
            self.dict_event = EVENT_DICTIONARY_V2
            self.num_classes = 17
            self.labels="Labels-v2.json"

        logging.info("Checking/Download features and labels locally")
        downloader = SoccerNetDownloader(path)
        downloader.downloadGames(files=[self.labels, f"1_{self.features}", f"2_{self.features}"], split=split, verbose=False,randomized=True)


        logging.info("Pre-compute clips")

        self.game_feats = list()
        self.game_labels = list()

        # game_counter = 0
        
        for game in tqdm(self.listGames):
            # Load features
            feat_half1 = np.load(os.path.join(self.path, game, "1_" + self.features))
            feat_half1 = feat_half1.reshape(-1, feat_half1.shape[-1])
            feat_half2 = np.load(os.path.join(self.path, game, "2_" + self.features))
            feat_half2 = feat_half2.reshape(-1, feat_half2.shape[-1])

            feat_half1 = feats2clip(torch.from_numpy(feat_half1), stride=self.window_size_frame, clip_length=self.window_size_frame)
            feat_half2 = feats2clip(torch.from_numpy(feat_half2), stride=self.window_size_frame, clip_length=self.window_size_frame)

            # Load labels
            labels = json.load(open(os.path.join(self.path, game, self.labels)))

            label_half1 = np.zeros((feat_half1.shape[0], self.num_classes+1))
            label_half1[:,0]=1 # those are BG classes
            label_half2 = np.zeros((feat_half2.shape[0], self.num_classes+1))
            label_half2[:,0]=1 # those are BG classes


            for annotation in labels["annotations"]:

                time = annotation["gameTime"]
                event = annotation["label"]

                half = int(time[0])

                minutes = int(time[-5:-3])
                seconds = int(time[-2::])
                frame = framerate * ( seconds + 60 * minutes ) 

                if version == 1:
                    if "card" in event: label = 0
                    elif "subs" in event: label = 1
                    elif "soccer" in event: label = 2
                    else: continue
                elif version == 2:
                    if event not in self.dict_event:
                        continue
                    label = self.dict_event[event]

                # if label outside temporal of view
                if half == 1 and frame//self.window_size_frame>=label_half1.shape[0]:
                    continue
                if half == 2 and frame//self.window_size_frame>=label_half2.shape[0]:
                    continue

                if half == 1:
                    label_half1[frame//self.window_size_frame][0] = 0 # not BG anymore
                    label_half1[frame//self.window_size_frame][label+1] = 1 # that's my class

                if half == 2:
                    label_half2[frame//self.window_size_frame][0] = 0 # not BG anymore
                    label_half2[frame//self.window_size_frame][label+1] = 1 # that's my class
            
            self.game_feats.append(feat_half1)
            self.game_feats.append(feat_half2)
            self.game_labels.append(label_half1)
            self.game_labels.append(label_half2)

        self.game_feats = np.concatenate(self.game_feats)
        self.game_labels = np.concatenate(self.game_labels)



    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            clip_feat (np.array): clip of features.
            clip_labels (np.array): clip of labels for the segmentation.
            clip_targets (np.array): clip of targets for the spotting.
        """
        return self.game_feats[index,:,:], self.game_labels[index,:]

    def __len__(self):
        return len(self.game_feats)


class SoccerNetClipsTesting(Dataset):
    def __init__(self, path, features="baidu_soccer_embeddings.npy", split=["test"], version=2, 
                framerate=1, window_size=10):
        self.path = path
        self.listGames = getListGames(split)
        self.features = features
        self.window_size_frame = window_size*framerate
        self.framerate = framerate
        self.version = version
        self.split=split
        if version == 1:
            self.dict_event = EVENT_DICTIONARY_V1
            self.num_classes = 3
            self.labels="Labels.json"
        elif version == 2:
            self.dict_event = EVENT_DICTIONARY_V2
            self.num_classes = 17
            self.labels="Labels-v2.json"

        logging.info("Checking/Download features and labels locally")
        downloader = SoccerNetDownloader(path)
        for s in split:
            if s == "challenge":
                downloader.downloadGames(files=[f"1_{self.features}", f"2_{self.features}"], split=[s], verbose=False,randomized=True)
            else:
                downloader.downloadGames(files=[self.labels, f"1_{self.features}", f"2_{self.features}"], split=[s], verbose=False,randomized=True)


    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            feat_half1 (np.array): features for the 1st half.
            feat_half2 (np.array): features for the 2nd half.
            label_half1 (np.array): labels (one-hot) for the 1st half.
            label_half2 (np.array): labels (one-hot) for the 2nd half.
        """
        # Load features
        feat_half1 = np.load(os.path.join(self.path, self.listGames[index], "1_" + self.features))
        feat_half1 = feat_half1.reshape(-1, feat_half1.shape[-1])
        feat_half2 = np.load(os.path.join(self.path, self.listGames[index], "2_" + self.features))
        feat_half2 = feat_half2.reshape(-1, feat_half2.shape[-1])

        # Load labels
        label_half1 = np.zeros((feat_half1.shape[0], self.num_classes))
        label_half2 = np.zeros((feat_half2.shape[0], self.num_classes))

        # check if annoation exists
        if os.path.exists(os.path.join(self.path, self.listGames[index], self.labels)):
            labels = json.load(open(os.path.join(self.path, self.listGames[index], self.labels)))

            for annotation in labels["annotations"]:

                time = annotation["gameTime"]
                event = annotation["label"]

                half = int(time[0])

                minutes = int(time[-5:-3])
                seconds = int(time[-2::])
                frame = self.framerate * ( seconds + 60 * minutes ) 

                if self.version == 1:
                    if "card" in event: label = 0
                    elif "subs" in event: label = 1
                    elif "soccer" in event: label = 2
                    else: continue
                elif self.version == 2:
                    if event not in self.dict_event:
                        continue
                    label = self.dict_event[event]

                value = 1
                if "visibility" in annotation.keys():
                    if annotation["visibility"] == "not shown":
                        value = -1

                if half == 1:
                    frame = min(frame, feat_half1.shape[0]-1)
                    label_half1[frame][label] = value

                if half == 2:
                    frame = min(frame, feat_half2.shape[0]-1)
                    label_half2[frame][label] = value

        
            

        feat_half1 = feats2clip(torch.from_numpy(feat_half1), 
                        stride=1, off=int(self.window_size_frame/2), 
                        clip_length=self.window_size_frame)

        feat_half2 = feats2clip(torch.from_numpy(feat_half2), 
                        stride=1, off=int(self.window_size_frame/2), 
                        clip_length=self.window_size_frame)

        
        return self.listGames[index], feat_half1, feat_half2, label_half1, label_half2

    def __len__(self):
        return len(self.listGames)





Defining Loss

In [3]:
import torch


class NLLLoss(torch.nn.Module):
    def __init__(self):
        super(NLLLoss, self).__init__()

    def forward(self, labels, output):
        # return torch.mean(labels * -torch.log(output) + (1 - labels) * -torch.log(1 - output))

        return torch.mean(torch.mean(labels * -torch.log(output) + (1 - labels) * -torch.log(1 - output)))


Creating the Model

In [4]:
class NetRVLAD(nn.Module):
    def __init__(self, cluster_size, feature_size, add_batch_norm=True):
        super(NetRVLAD, self).__init__()
        self.feature_size = feature_size
        self.cluster_size = cluster_size
        self.clusters = nn.Parameter((1/math.sqrt(feature_size))
                *torch.randn(feature_size, cluster_size))
        # self.clusters2 = nn.Parameter((1/math.sqrt(feature_size))
        #         *th.randn(1, feature_size, cluster_size))
        # self.clusters = nn.Parameter(torch.rand(1,feature_size, cluster_size))
        # self.clusters2 = nn.Parameter(torch.rand(1,feature_size, cluster_size))

        self.add_batch_norm = add_batch_norm
        # self.batch_norm = nn.BatchNorm1d(cluster_size)
        self.out_dim = cluster_size*feature_size
        #  (+ 128 params?)
    def forward(self,x):
        max_sample = x.size()[1]

        # LOUPE
        if self.add_batch_norm: # normalization along feature dimension
            x = F.normalize(x, p=2, dim=2)

        x = x.reshape(-1,self.feature_size)
        assignment = torch.matmul(x,self.clusters)

        assignment = F.softmax(assignment,dim=1)
        assignment = assignment.view(-1, max_sample, self.cluster_size)

        # a_sum = th.sum(assignment,-2,keepdim=True)
        # a = a_sum*self.clusters2

        assignment = assignment.transpose(1,2)

        x = x.view(-1, max_sample, self.feature_size)
        rvlad = torch.matmul(assignment, x)
        rvlad = rvlad.transpose(-1,1)

        # vlad = vlad.transpose(1,2)
        # vlad = vlad - a

        # L2 intra norm
        rvlad = F.normalize(rvlad)
        
        # flattening + L2 norm
        rvlad = rvlad.reshape(-1, self.cluster_size*self.feature_size)
        rvlad = F.normalize(rvlad)

        return rvlad

Defining the Model

In [5]:
class Model(nn.Module):
    def __init__(self, weights=None, input_size=8576, num_classes=17, vocab_size=64, window_size=10, framerate=1, pool="NetRVLAD++"):
        """
        INPUT: a Tensor of shape (batch_size,window_size,feature_size)
        OUTPUTS: a Tensor of shape (batch_size,num_classes+1)
        """

        super(Model, self).__init__()

        self.window_size_frame=window_size * framerate
        self.input_size = input_size
        self.num_classes = num_classes
        self.framerate = framerate
        self.pool = pool
        self.vlad_k = vocab_size
        
        # are feature alread PCA'ed?
        if not self.input_size == 512:   
            self.feature_extractor = nn.Linear(self.input_size, 512)
            input_size = 512
            self.input_size = 512

        if self.pool == "MAX":
            self.pool_layer = nn.MaxPool1d(self.window_size_frame, stride=1)
            self.fc = nn.Linear(input_size, self.num_classes+1)

        if self.pool == "MAX++":
            self.pool_layer_before = nn.MaxPool1d(int(self.window_size_frame/2), stride=1)
            self.pool_layer_after = nn.MaxPool1d(int(self.window_size_frame/2), stride=1)
            self.fc = nn.Linear(2*input_size, self.num_classes+1)


        if self.pool == "AVG":
            self.pool_layer = nn.AvgPool1d(self.window_size_frame, stride=1)
            self.fc = nn.Linear(input_size, self.num_classes+1)

        if self.pool == "AVG++":
            self.pool_layer_before = nn.AvgPool1d(int(self.window_size_frame/2), stride=1)
            self.pool_layer_after = nn.AvgPool1d(int(self.window_size_frame/2), stride=1)
            self.fc = nn.Linear(2*input_size, self.num_classes+1)


        elif self.pool == "NetVLAD":
            self.pool_layer = NetVLAD(cluster_size=self.vlad_k, feature_size=self.input_size,
                                            add_batch_norm=True)
            self.fc = nn.Linear(input_size*self.vlad_k, self.num_classes+1)

        elif self.pool == "NetVLAD++":
            self.pool_layer_before = NetVLAD(cluster_size=int(self.vlad_k/2), feature_size=self.input_size,
                                            add_batch_norm=True)
            self.pool_layer_after = NetVLAD(cluster_size=int(self.vlad_k/2), feature_size=self.input_size,
                                            add_batch_norm=True)
            self.fc = nn.Linear(input_size*self.vlad_k, self.num_classes+1)



        elif self.pool == "NetRVLAD":
            self.pool_layer = NetRVLAD(cluster_size=self.vlad_k, feature_size=self.input_size,
                                            add_batch_norm=True)
            self.fc = nn.Linear(input_size*self.vlad_k, self.num_classes+1)

        elif self.pool == "NetRVLAD++":
            self.pool_layer_before = NetRVLAD(cluster_size=int(self.vlad_k/2), feature_size=self.input_size,
                                            add_batch_norm=True)
            self.pool_layer_after = NetRVLAD(cluster_size=int(self.vlad_k/2), feature_size=self.input_size,
                                            add_batch_norm=True)
            self.fc = nn.Linear(input_size*self.vlad_k, self.num_classes+1)

        self.drop = nn.Dropout(p=0.4)
        self.sigm = nn.Sigmoid()

        self.load_weights(weights=weights)

    def load_weights(self, weights=None):
        if(weights is not None):
            print("=> loading checkpoint '{}'".format(weights))
            checkpoint = torch.load(weights)
            self.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(weights, checkpoint['epoch']))

    def forward(self, inputs):
        # input_shape: (batch,frames,dim_features)


        BS, FR, IC = inputs.shape
        if not IC == 512:
            inputs = inputs.reshape(BS*FR, IC)
            inputs = self.feature_extractor(inputs)
            inputs = inputs.reshape(BS, FR, -1)

        # Temporal pooling operation
        if self.pool == "MAX" or self.pool == "AVG":
            inputs_pooled = self.pool_layer(inputs.permute((0, 2, 1))).squeeze(-1)

        elif self.pool == "MAX++" or self.pool == "AVG++":
            nb_frames_50 = int(inputs.shape[1]/2)    
            input_before = inputs[:, :nb_frames_50, :]        
            input_after = inputs[:, nb_frames_50:, :]  
            inputs_before_pooled = self.pool_layer_before(input_before.permute((0, 2, 1))).squeeze(-1)
            inputs_after_pooled = self.pool_layer_after(input_after.permute((0, 2, 1))).squeeze(-1)
            inputs_pooled = torch.cat((inputs_before_pooled, inputs_after_pooled), dim=1)


        elif self.pool == "NetVLAD" or self.pool == "NetRVLAD":
            inputs_pooled = self.pool_layer(inputs)

        elif self.pool == "NetVLAD++" or self.pool == "NetRVLAD++":
            nb_frames_50 = int(inputs.shape[1]/2)
            inputs_before_pooled = self.pool_layer_before(inputs[:, :nb_frames_50, :])
            inputs_after_pooled = self.pool_layer_after(inputs[:, nb_frames_50:, :])
            inputs_pooled = torch.cat((inputs_before_pooled, inputs_after_pooled), dim=1)


        # Extra FC layer with dropout and sigmoid activation
        output = self.sigm(self.fc(self.drop(inputs_pooled)))

        return output


BS =256
T = 15
framerate= 2
D = 512
pool = "NetRVLAD++"
model = Model()
print(model)
inp = torch.rand([BS,T*framerate,D])
print(inp.shape)
output = model(inp)
print(output.shape)

Model(
  (feature_extractor): Linear(in_features=8576, out_features=512, bias=True)
  (pool_layer_before): NetRVLAD()
  (pool_layer_after): NetRVLAD()
  (fc): Linear(in_features=32768, out_features=18, bias=True)
  (drop): Dropout(p=0.4, inplace=False)
  (sigm): Sigmoid()
)
torch.Size([256, 30, 512])
torch.Size([256, 18])


Training the Model

In [6]:
def trainer(train_loader,
            val_loader,
            val_metric_loader,
            model,
            optimizer,
            #scheduler,
            criterion,
            #model_name,
            max_epochs=10,
            evaluation_frequency=2):

    logging.info("start training")

    best_loss = 9e99

    for epoch in range(max_epochs):
        #best_model_path = os.path.join("models", model_name, "model.pth.tar")

        # train for one epoch
        loss_training = train(train_loader, model, criterion,
                              optimizer, epoch + 1, train=True)

        # evaluate on validation set
        loss_validation = train(
            val_loader, model, criterion, optimizer, epoch + 1, train=False)

        #state = {
        #    'epoch': epoch + 1,
        #    'state_dict': model.state_dict(),
        #    'best_loss': best_loss,
        #    'optimizer': optimizer.state_dict(),
        #}
        #os.makedirs(os.path.join("models", model_name), exist_ok=True)

        # remember best prec@1 and save checkpoint
        #is_better = loss_validation < best_loss
        #best_loss = min(loss_validation, best_loss)

        # Save the best model based on loss only if the evaluation frequency too long
        #if is_better:
        #    torch.save(state, best_model_path)

        # Test the model on the validation set
        if epoch % evaluation_frequency == 0 and epoch != 0:
            performance_validation = test(
                val_metric_loader,
                model)

            print("Validation performance at epoch " +
                         str(epoch+1) + " -> " + str(performance_validation))

        # Reduce LR on Plateau after patience reached
        #prevLR = optimizer.param_groups[0]['lr']
        #scheduler.step(loss_validation)
        #currLR = optimizer.param_groups[0]['lr']
        #if (currLR is not prevLR and scheduler.num_bad_epochs == 0):
        #    logging.info("Plateau Reached!")

        #if (prevLR < 2 * scheduler.eps and
        #        scheduler.num_bad_epochs >= scheduler.patience):
        #    logging.info(
        #        "Plateau Reached and no more reduction -> Exiting Loop")
        #    break

    return


def train(dataloader,
          model,
          criterion,
          optimizer,
          epoch,
          train=False):

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    # switch to train mode
    if train:
        model.train()
    else:
        model.eval()

    end = time.time()
    with tqdm(enumerate(dataloader), total=len(dataloader)) as t:
        for i, (feats, labels) in t:
            # measure data loading time
            data_time.update(time.time() - end)
            feats = feats.cuda()
            labels = labels.cuda()
            # compute output
            output = model(feats)

            # hand written NLL criterion
            loss = criterion(labels, output)

            # measure accuracy and record loss
            losses.update(loss.item(), feats.size(0))

            if train:
                # compute gradient and do SGD step
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if train:
                desc = f'Train {epoch}: '
            else:
                desc = f'Evaluate {epoch}: '
            desc += f'Time {batch_time.avg:.3f}s '
            desc += f'(it:{batch_time.val:.3f}s) '
            desc += f'Data:{data_time.avg:.3f}s '
            desc += f'(it:{data_time.val:.3f}s) '
            desc += f'Loss {losses.avg:.4e} '
            t.set_description(desc)

    return losses.avg

def test(dataloader, model):
    batch_time = AverageMeter()
    data_time = AverageMeter()

    model.eval()

    end = time.time()
    all_labels = []
    all_outputs = []
    with tqdm(enumerate(dataloader), total=len(dataloader)) as t:
        for i, (feats, labels) in t:
            # measure data loading time
            data_time.update(time.time() - end)
            feats = feats.cuda()
            # labels = labels.cuda()

            # print(feats.shape)
            # feats=feats.unsqueeze(0)
            # print(feats.shape)

            # compute output
            output = model(feats)

            all_labels.append(labels.detach().numpy())
            all_outputs.append(output.cpu().detach().numpy())

            batch_time.update(time.time() - end)
            end = time.time()

            desc = f'Test (cls): '
            desc += f'Time {batch_time.avg:.3f}s '
            desc += f'(it:{batch_time.val:.3f}s) '
            desc += f'Data:{data_time.avg:.3f}s '
            desc += f'(it:{data_time.val:.3f}s) '
            t.set_description(desc)

    AP = []
    for i in range(1, dataloader.dataset.num_classes+1):
        AP.append(average_precision_score(np.concatenate(all_labels)
                                          [:, i], np.concatenate(all_outputs)[:, i]))

    # t.set_description()
    # print(AP)
    mAP = np.mean(AP)
    print(mAP, AP)

    return mAP

Executing the Model

In [7]:
soccernet_path = "drive/MyDrive/SoccerNetDatasetFeatures/"

dataset_Train = SoccerNetClips(path=soccernet_path)
dataset_Valid = SoccerNetClips(path=soccernet_path,  split=["valid"])
dataset_Valid_metric  = SoccerNetClips(path=soccernet_path, split=["valid"])


model = Model().cuda()
logging.info(model)
total_params = sum(p.numel()
                       for p in model.parameters() if p.requires_grad)
logging.info("Total number of parameters: " + str(total_params))


train_loader = torch.utils.data.DataLoader(dataset_Train,
            batch_size=64, shuffle=True,
            num_workers=4, pin_memory=True)

val_loader = torch.utils.data.DataLoader(dataset_Valid,
            batch_size=64, shuffle=False,
            num_workers=4, pin_memory=True)

val_metric_loader = torch.utils.data.DataLoader(dataset_Valid_metric,
            batch_size=64, shuffle=False,
            num_workers=4, pin_memory=True)

criterion = NLLLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, 
                                    betas=(0.9, 0.999), eps=1e-08, 
                                    weight_decay=0, amsgrad=False)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', verbose=True)
trainer(train_loader, val_loader, val_metric_loader, model, optimizer, criterion)



100%|██████████| 50/50 [00:22<00:00,  2.21it/s]
Train 1: Time 0.022s (it:0.011s) Data:0.008s (it:0.003s) Loss 4.9423e-01 : 100%|██████████| 434/434 [00:08<00:00, 51.57it/s]
Evaluate 1: Time 0.018s (it:0.010s) Data:0.013s (it:0.006s) Loss 3.5157e-01 : 100%|██████████| 430/430 [00:06<00:00, 65.97it/s]
Train 2: Time 0.021s (it:0.008s) Data:0.007s (it:0.001s) Loss 2.8207e-01 : 100%|██████████| 434/434 [00:08<00:00, 53.15it/s]
Evaluate 2: Time 0.018s (it:0.009s) Data:0.013s (it:0.006s) Loss 2.2924e-01 : 100%|██████████| 430/430 [00:06<00:00, 65.15it/s]
Train 3: Time 0.021s (it:0.009s) Data:0.007s (it:0.001s) Loss 2.0182e-01 : 100%|██████████| 434/434 [00:08<00:00, 53.31it/s]
Evaluate 3: Time 0.018s (it:0.010s) Data:0.013s (it:0.007s) Loss 1.7836e-01 : 100%|██████████| 430/430 [00:06<00:00, 64.73it/s]
Test (cls): Time 0.026s (it:0.009s) Data:0.013s (it:0.006s) : 100%|██████████| 430/430 [00:10<00:00, 42.46it/s]

0.12831096784964338 [0.14641986031707532, 0.016188682089266423, 0.0898654572815745, 0.020970071480460168, 0.016746353490021893, 0.12566975088350002, 0.16556310129040708, 0.04943329645341987, 0.608699086701906, 0.4115862205791728, 0.08107597310314314, 0.0471292884375216, 0.08903546748040077, 0.2868891619362755, 0.02534792267892492, 0.0004494247088141252, 0.00021733453205330483]
Validation performance at epoch 3 -> 0.12831096784964338



Train 4: Time 0.021s (it:0.008s) Data:0.007s (it:0.001s) Loss 1.6593e-01 : 100%|██████████| 434/434 [00:08<00:00, 53.07it/s]
Evaluate 4: Time 0.018s (it:0.010s) Data:0.012s (it:0.007s) Loss 1.5316e-01 : 100%|██████████| 430/430 [00:06<00:00, 64.35it/s]
Train 5: Time 0.021s (it:0.009s) Data:0.007s (it:0.001s) Loss 1.4522e-01 : 100%|██████████| 434/434 [00:08<00:00, 52.71it/s]
Evaluate 5: Time 0.018s (it:0.010s) Data:0.012s (it:0.006s) Loss 1.3519e-01 : 100%|██████████| 430/430 [00:06<00:00, 64.97it/s]
Test (cls): Time 0.018s (it:0.011s) Data:0.013s (it:0.007s) : 100%|██████████| 430/430 [00:06<00:00, 64.85it/s]

0.11473765302152755 [0.18957168906929167, 0.012343942553247955, 0.15569785087069377, 0.016486632128230344, 0.019549075263905816, 0.1467941010137189, 0.1794340843774903, 0.03882775363260976, 0.5281109260665895, 0.258028116920993, 0.07337056542131452, 0.03646675868056548, 0.11218219254343774, 0.16899589509973822, 0.014184044510485248, 0.0003746192748344654, 0.00012185393882163392]
Validation performance at epoch 5 -> 0.11473765302152755



Train 6: Time 0.021s (it:0.009s) Data:0.007s (it:0.001s) Loss 1.2893e-01 : 100%|██████████| 434/434 [00:08<00:00, 52.54it/s]
Evaluate 6: Time 0.018s (it:0.010s) Data:0.012s (it:0.006s) Loss 1.2109e-01 : 100%|██████████| 430/430 [00:06<00:00, 63.63it/s]
Train 7: Time 0.021s (it:0.009s) Data:0.007s (it:0.002s) Loss 1.1648e-01 : 100%|██████████| 434/434 [00:08<00:00, 53.31it/s]
Evaluate 7: Time 0.018s (it:0.011s) Data:0.012s (it:0.007s) Loss 1.1077e-01 : 100%|██████████| 430/430 [00:06<00:00, 64.08it/s]
Test (cls): Time 0.018s (it:0.010s) Data:0.013s (it:0.006s) : 100%|██████████| 430/430 [00:06<00:00, 64.98it/s]

0.11563085227170294 [0.27484114810533256, 0.017985278607480726, 0.17741658213160208, 0.02589640009887169, 0.020536788448312686, 0.13112600658380563, 0.13985208213247916, 0.04750437512770179, 0.5027884102962255, 0.24950720646529462, 0.09677578082045077, 0.045686875580536405, 0.09427575867562443, 0.1172964070537906, 0.02328975653804538, 0.0008189439835268879, 0.00012668796986902377]
Validation performance at epoch 7 -> 0.11563085227170294



Train 8: Time 0.021s (it:0.010s) Data:0.007s (it:0.003s) Loss 1.0742e-01 : 100%|██████████| 434/434 [00:08<00:00, 52.76it/s]
Evaluate 8: Time 0.018s (it:0.010s) Data:0.012s (it:0.006s) Loss 1.0340e-01 : 100%|██████████| 430/430 [00:06<00:00, 64.21it/s]
Train 9: Time 0.021s (it:0.008s) Data:0.007s (it:0.001s) Loss 1.0082e-01 : 100%|██████████| 434/434 [00:08<00:00, 52.95it/s]
Evaluate 9: Time 0.018s (it:0.010s) Data:0.012s (it:0.006s) Loss 9.8059e-02 : 100%|██████████| 430/430 [00:06<00:00, 64.28it/s]
Test (cls): Time 0.018s (it:0.011s) Data:0.013s (it:0.008s) : 100%|██████████| 430/430 [00:06<00:00, 63.95it/s]

0.11955290569638144 [0.24436854201120714, 0.017561968281903352, 0.1822926087404041, 0.02184269469500308, 0.021444864957994175, 0.13567074377026234, 0.14731715639873516, 0.051013925645895276, 0.5446113679401267, 0.25992888853013824, 0.10908629109238667, 0.05272532450841179, 0.09099953970047418, 0.1275007656846865, 0.02443949180998853, 0.0014469854858267645, 0.00014823758504043446]
Validation performance at epoch 9 -> 0.11955290569638144



Train 10: Time 0.021s (it:0.009s) Data:0.007s (it:0.002s) Loss 9.5811e-02 : 100%|██████████| 434/434 [00:08<00:00, 52.85it/s]
Evaluate 10: Time 0.018s (it:0.011s) Data:0.012s (it:0.006s) Loss 9.4129e-02 : 100%|██████████| 430/430 [00:06<00:00, 64.49it/s]


In [8]:
print(torch.cuda.is_available())

True
